In [131]:
import pandas as pd
import numpy as np

In [132]:
year_list = range(2011,2017,1)
cpi = [114.51, 117.76, 119.07, 119.48, 120.15, 120.15]
income_list = ["earns","dispy","tax","sicer","sicee","sicse","ben","origy"]
data_path = "data/"

In [133]:
# read and merge data

silc = pd.read_table(data_path+"be_2012_a4.txt",sep='\t')
#print(silc.shape)

# for each year in year_list
for year in year_list:
    # read the tab seperated file
    df = pd.read_table(data_path+"be_%d_std.txt" %year,sep='\t')
    # keep only id and columns starting with 'ils'
    columns = df.columns
    keep_columns = ["idperson"] + list(columns[columns.str.startswith("ils_")])
    df = df[keep_columns]
    # add suffix of the year to all the column names except idperson
    columns = pd.Series(df.columns)
    columns[columns.str.startswith("ils_")] = columns[columns.str.startswith("ils_")] + "_%d" %year
    df.columns = columns
    # merge with silc on idperson
    silc = silc.merge(df,on="idperson")
    #print(silc.shape)

In [134]:
## GENERATE ADDITIONAL VARIABLES AT INDIVIDUAL LEVEL

# gross income for calculating household head later
silc['tmp_gr_inc'] = -1000 * (silc['dag']<18) + silc['ils_origy_2011'] + 0.0001*np.random.uniform(size=silc.shape[0])
    # make sure no minor becomes hhh
    #it happens that people in the hh have same gross income -> add random very small amount

# indicator for underage 
silc['ch'] = silc['dag']<18

# socio-economic status
silc['working']= (silc['dag'] >= 18) & ( (silc['yem'] + silc['yse']) > 200 ) 
silc['pension']= (silc['dag']  >= 40) & (silc['poa'] > (silc['yem'] + silc['yse'] + silc['bun'])) & (silc['working'] == False)
silc['unempl']= (silc['dag']  >= 18) & ( (silc['bun']>0) | (silc['les'] == 5) ) & (silc['working'] == False) & (silc['pension'] == False)
silc['inactive']= (silc['dag']  >= 18) & (silc['les'] != 6) & (silc['unempl'] == 0) & (silc['working'] == 0) & (silc['pension'] == False)

In [135]:
## AGGREGATIONS AT HOUSEHOLD LEVEL

# create dictionary to store aggregations at the household level
hh_agg_dict = {}

for year in year_list:
    for var in income_list:
        #print("ils_%s_%s" %(var,year))
        hh_agg_dict["ils_%s_%s" %(var,year)] = {"hh_%s_%s" %(var,year):"sum"}

hh_agg_dict['idperson'] = {'hh_size': 'count'}
hh_agg_dict['ch'] = {'hh_ch':'sum'}
hh_agg_dict['tmp_gr_inc'] = {'tmp_gr_max':'max'}

# create household measures by grouping and using aggregation dictionary
hh_data = silc.groupby('idhh').agg(hh_agg_dict)

# drop upper level of multiindex
hh_data.columns = hh_data.columns.droplevel(0)

# merge household measures back to individual measures
silc = silc.merge(hh_data,left_on='idhh',right_index=True)

In [136]:
# indicator for household head
silc['hhh'] = silc['tmp_gr_inc'] == silc['tmp_gr_max']

In [137]:
## EQUIVALENCE

# create equivalence scale
tmp = 1 * silc['hhh']==1  #assign 1 to the oldest person in the hh
tmp[(silc['dag']>=14) & (tmp==0)] = 0.5 # other adults 0.5
tmp[(silc['dag'] < 14)  & (tmp == 0)] = 0.3 # and children 0.3

# calculate sum of equivalence scale by household
silc['tmp'] = tmp
eq_scale = silc.groupby('idhh')['tmp'].sum()

# merge sum back to silc
silc = silc.merge(pd.DataFrame(data=eq_scale).rename(columns={"tmp":"eq_scale"}),left_on='idhh',right_index=True)

# create equivalent disposable income
for year in year_list:
    silc['eq_dispy_%s' %year] = silc['hh_dispy_%s' %year] / silc['eq_scale']

In [138]:
# weights corrected for household size
silc['w_hh'] = silc['dwt'] * silc['hh_size']
silc['w_hh'] = silc['w_hh'].astype(int)

In [139]:
silc.to_pickle("data/be_em_2011_2016")

In [140]:
silc = pd.read_pickle("data/be_em_2011_2016")

## Function for computing weighted quantiles

In [141]:
# function for weighted quantiles from http://stackoverflow.com/questions/21844024/weighted-percentile-using-numpy

def weighted_quantile(values, quantiles, sample_weight=None, values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of initial array
    :param old_style: if True, will correct output to be consistent with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), 'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

## Prep for groups

In [142]:
# construct deciles of individuals in the population ranked by equivalised disposable household income

# create empty dictionary that will store the quantiles as lists as values and the groups as keys
w_quantiles_by_group = {}

# keep frequency weights of rows with household heads
silc_hh = silc[silc["hhh"]==True]

## Create subgroups (deciles, ventiles, etc)

In [187]:
n_quantile = 5 # number of quantiles
var_q = "hh_origy_2011" # variable to be used in ordering
var_x = "hh_origy_2011" # variable on x-axis
var_y = "hh_dispy_2011" # variable on y-axis
group = "drgn1"# grouping over a factor variable (drgn1 = region)

In [180]:
n_quantile = 10 # number of quantiles
var_q = "eq_dispy_2011" # variable to be used in ordering
var_x = "hh_dispy_2011" # variable on x-axis
var_y = "drgn1" # variable on y-axis
group = "drgn1"# grouping over a factor variable (drgn1 = region)

In [188]:
# make list of quantiles to compute
my_quantiles = [x/float(100) for x in range(0,101,100/n_quantile)]

In [189]:
# for each group in the grouping variable
for groupi in np.unique(silc[group]):
    # keep only the lines of silc_hh from that group
    silc_use = silc_hh[(silc_hh[group]==groupi)]
    # calculate the weighted quantiles and store in the dictionary
    w_quantiles_by_group[groupi] = weighted_quantile(values = silc_use[var_q],
                                                   quantiles = my_quantiles, 
                                                   sample_weight = silc_use["w_hh"])
    # store quantiles in silc
    silc.loc[(silc[group]==groupi),"w_quantile"] = pd.cut(silc[var_q],bins=w_quantiles_by_group[groupi])

In [190]:
w_quantiles_by_group

{1: array([ -1.01667000e+03,   2.50000000e+00,   1.17590413e+03,
          2.79745178e+03,   5.09748491e+03,   3.46780400e+04]),
 2: array([ -1983.33      ,    103.07      ,   2746.37421964,   4529.17803572,
          6527.52556355,  35495.42      ]),
 3: array([ -1.64641000e+03,   7.08000000e+00,   1.96316709e+03,
          3.62103548e+03,   5.72148788e+03,   2.78379500e+04])}

## CHART

In [191]:
import matplotlib.pyplot as plt
%matplotlib inline

In [192]:
data_to_plot = silc.ix[silc["hhh"]==True,np.unique([var_y,var_x,var_q,group,"w_quantile"])].groupby(["w_quantile",group]).mean().reset_index()
data_to_plot = data_to_plot.sort_values([var_x,group])
data_to_plot.head(6)

,w_quantile,drgn1,hh_dispy_2011,hh_origy_2011
0,"(-1016.67, 2.5]",1,1261.358940,-28.094516
1,"(-1646.41, 7.08]",3,1508.582480,-13.169705
2,"(-1983.33, 103.07]",2,1638.595201,0.556406
6,"(2.5, 1175.904]",1,1481.726383,354.848830
14,"(7.08, 1963.167]",3,1928.775551,697.048423
3,"(103.07, 2746.374]",2,2081.829811,1451.499956


In [193]:
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div, CustomJS, Range1d, CategoricalColorMapper
from bokeh.models.widgets import Slider, Select, TextInput, CheckboxGroup
from bokeh.io import curdoc
from bokeh.palettes import *

groups = np.unique(silc[group])

# Create Column Data Source that will be used by the plot
source = ColumnDataSource(data=data_to_plot)
sources = {}
for groupi in groups:
    sources[groupi] = ColumnDataSource(data=data_to_plot[data_to_plot[group]==groupi])

my_palette = Category10[len(groups)]

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=list(np.unique(silc[group])), palette=my_palette)


# create plot
p = figure(plot_height=400, plot_width=600, title="", toolbar_location=None)
p.circle(x=var_x, y=var_y, source=source, color=dict(field=group, transform=color_mapper), legend=group)
for i in range(len(groups)):
    p.line(x=var_x, y=var_y, source=sources[groups[i]],color=my_palette[i])


# Set the legend.location attribute of the plot to 'top_right'
p.legend.location = 'top_right'


# add all the elements to a layout
#l = layout([p])

from bokeh.io import output_notebook, show
output_notebook()
show(p)

Loading BokehJS ...